In [1]:
# Componentes básicos da rede
from components import Network, Controller
# Funções auxiliares para executar as aplicações
from components.qkd.app import *

import numpy as np

In [2]:
# Criando a rede
rede = Network()

# Criando e definindo um controlador pra rede
controlador = Controller()
controlador.set_network(rede)

In [3]:
#  ________________________________________________________________
# |Simulação: Várias execuções                                     |
# |          _______________________________________________       |
# |         |Execução: geração e atendimento dos requests   |      |
# |         |                    ___________________________|      |
# |         |                   |Requests: apps QKD         |      |
# |         |___________________|___________________________|      |
# |________________________________________________________________|
#

In [4]:
# Propriedades para simulação
n_simulacoes = 100
n_requests = 100
n_total_execucoes = 0

## Coleta dos resultados

In [5]:
# Taxa de sucesso de bits das chaves
taxas_sucesso_chaves_geral = []
taxas_sucesso_chaves_e91 = []
taxas_sucesso_chaves_bb84 = []
taxas_sucesso_chaves_b92 = []

# Médias de sucesso de bits das chaves
medias_sucesso_chaves_geral = dict()


# Vazão da rede
vazao = []
media_vazao = []

In [6]:
# Executa num_simulacoes vezes simulações de requests
for indice, simulacao in enumerate(range(n_simulacoes)):
    
    taxas_sucesso_chaves_e91 = []
    taxas_sucesso_chaves_bb84 = []
    taxas_sucesso_chaves_b92 = []
    rede.set_fully_connected_topology(12)
        
    requests = generate_qkd_request(rede, n_requests)
    resultados_simulacao = controlador.send_requests(requests, 'kshortest')
    
    for indice_execucao in resultados_simulacao:
        # Percorre cada execução e colhe o sucesso nas chaves
        resultado_individual_simulacao = resultados_simulacao[indice_execucao]
        sucesso_chave = resultado_individual_simulacao['key sucess']
        
        # Colhendo o sucesso nas chaves de cada app (Tamamnho da chave recebida/ Tamanho da chave enviada)
        if resultado_individual_simulacao['app'] == 'BB84':
            taxas_sucesso_chaves_bb84.append(sucesso_chave)
            media_sucesso_chaves_bb84 = sum(taxas_sucesso_chaves_bb84) / len(taxas_sucesso_chaves_bb84)
        
        elif resultado_individual_simulacao['app'] == 'E91':
            taxas_sucesso_chaves_e91.append(sucesso_chave)
            media_sucesso_chaves_e91 = sum(taxas_sucesso_chaves_e91) / len(taxas_sucesso_chaves_e91)
            
        elif resultado_individual_simulacao['app'] == 'B92':
            taxas_sucesso_chaves_b92.append(sucesso_chave)
            media_sucesso_chaves_b92 = sum(taxas_sucesso_chaves_b92) / len(taxas_sucesso_chaves_b92)
      
    
    # Salvando o sucesso nas chaves geral da simulação
    lista_combinada = [taxa for sublist in [taxas_sucesso_chaves_bb84, taxas_sucesso_chaves_e91, taxas_sucesso_chaves_b92] for taxa in sublist]
    #print(lista_combinada)
    taxas_sucesso_chaves_geral.append(sum(lista_combinada) / len(lista_combinada))
    
    # Calculando a vazão
    n_execucoes = len(resultados_simulacao)
    vazao.append(n_requests / n_execucoes)
    

Rotas calculada [[11, 4], [11, 0, 4], [11, 1, 4], [11, 2, 4], [11, 3, 4]]
Rota avaliada [11, 4]
Rotas calculada [[3, 8], [3, 0, 8], [3, 1, 8], [3, 2, 8], [3, 4, 8]]
Rota avaliada [3, 8]
Rotas calculada [[9, 2], [9, 0, 2], [9, 1, 2], [9, 3, 2], [9, 4, 2]]
Rota avaliada [9, 2]
Rotas calculada [[8, 1], [8, 0, 1], [8, 2, 1], [8, 3, 1], [8, 4, 1]]
Rota avaliada [8, 1]
Rotas calculada [[6, 11], [6, 0, 11], [6, 1, 11], [6, 2, 11], [6, 3, 11]]
Rota avaliada [6, 11]
Rotas calculada [[6, 11], [6, 0, 11], [6, 1, 11], [6, 2, 11], [6, 3, 11]]
Rota avaliada [6, 11]
Rota avaliada [6, 0, 11]
Rotas calculada [[7, 0], [7, 1, 0], [7, 2, 0], [7, 3, 0], [7, 4, 0]]
Rota avaliada [7, 0]
Rotas calculada [[3, 8], [3, 0, 8], [3, 1, 8], [3, 2, 8], [3, 4, 8]]
Rota avaliada [3, 8]
Rota avaliada [3, 0, 8]
Rotas calculada [[3, 8], [3, 0, 8], [3, 1, 8], [3, 2, 8], [3, 4, 8]]
Rota avaliada [3, 8]
Rota avaliada [3, 0, 8]
Rota avaliada [3, 1, 8]
Rotas calculada [[1, 6], [1, 0, 6], [1, 2, 6], [1, 3, 6], [1, 4, 6]]
Rota a

Rotas calculada [[3, 8], [3, 0, 8], [3, 1, 8], [3, 2, 8], [3, 4, 8]]
Rota avaliada [3, 8]
Rota avaliada [3, 0, 8]
Rota avaliada [3, 1, 8]
Rota avaliada [3, 2, 8]
Rotas calculada [[6, 11], [6, 0, 11], [6, 1, 11], [6, 2, 11], [6, 3, 11]]
Rota avaliada [6, 11]
Rota avaliada [6, 0, 11]
Rotas calculada [[0, 5], [0, 1, 5], [0, 2, 5], [0, 3, 5], [0, 4, 5]]
Rota avaliada [0, 5]
Rota avaliada [0, 1, 5]
Rota avaliada [0, 2, 5]
Rota avaliada [0, 3, 5]
Rota avaliada [0, 4, 5]
Rotas calculada [[1, 6], [1, 0, 6], [1, 2, 6], [1, 3, 6], [1, 4, 6]]
Rota avaliada [1, 6]
Rotas calculada [[2, 7], [2, 0, 7], [2, 1, 7], [2, 3, 7], [2, 4, 7]]
Rota avaliada [2, 7]
Rota avaliada [2, 0, 7]
Rota avaliada [2, 1, 7]
Rota avaliada [2, 3, 7]
Rotas calculada [[10, 3], [10, 0, 3], [10, 1, 3], [10, 2, 3], [10, 4, 3]]
Rota avaliada [10, 3]
Rota avaliada [10, 0, 3]
Rota avaliada [10, 1, 3]
Rota avaliada [10, 2, 3]
Rota avaliada [10, 4, 3]
Rotas calculada [[11, 4], [11, 0, 4], [11, 1, 4], [11, 2, 4], [11, 3, 4]]
Rota aval

In [7]:
print(np.mean(vazao))

26.783333333333335


In [8]:
print(np.mean(taxas_sucesso_chaves_geral))

0.2585166666666667


In [9]:
print(np.std(taxas_sucesso_chaves_geral))

0.08316646459561289
